# First Quantized Hamiltonian with Quantum Projectile

SELECT and PREPARE for the first quantized chemistry Hamiltonian.

Here we assume the Born-Oppenheimer Hamiltonian and seek to simulate a
collection of $\eta$ electrons and $L$ static nuclei with a Hamiltonian given
by:
$$
H_{BO} = T + U + V + \frac{1}{2}
\sum_{\ell\ne\kappa=1}^L\frac{\zeta_\ell\zeta_\kappa}{\lVert R_\ell - R_\kappa \rVert}
$$

In the first quantized approach we assume periodic boundary conditions and use a
plane wave Galerkin discretization.
A plane wave basis function is given by
$$
\phi_p(r) = \frac{1}{\sqrt{\Omega}} e^{-i k_p\cdot r}
$$
where $r$ is a position vector in real space, $\Omega$ is the simulation cell
volume and $k_p$ is a reciprocal lattice vector.
In three dimensions we have
$$
k_p = \frac{2\pi p }{\Omega}
$$
for $p \in G$ and
$$
G = [-\frac{N^{1/3} -
1}{2},\frac{N^{1/3} - 1}{2}]^3 \subset \mathcal{Z}^3.
$$
and $N$ is the total number of planewaves.

With these definitions we can write the components of the Hamiltonian as:
$$
T = \sum_{i}^\eta\sum_{p\in G}\frac{\lVert k_p\rVert^2}{2} |p\rangle\langle p|_i
$$
which defines the kinetic energy of the electrons,
$$
U = -\frac{4\pi}{\Omega}
\sum_{\ell=1}^L \sum_{i}^\eta
\sum_{p,q\in G, p\ne q}
\left(
    \zeta_{\ell}
    \frac{e^{i k_{q-p}\cdot R_\ell}}{\lVert k_{p-q}\rVert^2}
    |p\rangle\langle q|_i
\right)
$$
describes the interaction of the electrons and the nuclei, and,
$$
V = \frac{2\pi}{\Omega}
\sum_{i\ne j=1}^\eta
\sum_{p,q\in G, p\ne q}
\sum_{\nu \in G_0}
\left(
    \frac{1}{\lVert k_{\nu}\rVert^2}
    |p + \nu\rangle\langle p|_i
    |q -\nu\rangle\langle q|_i
\right)
$$
describes the electron-electron interaction. The notation $|p\rangle\langle p|_i$ is shorthand for
$I_1\otimes\cdots\otimes |p\rangle \langle p |_j \otimes \cdots \otimes I_\eta$.
The system is represented using a set of $\eta$ signed integer registers each of
size $3 n_p$ where $n_p =  \lceil \log (N^{1/3} + 1) \rceil$, with the factor of
3 accounting for the 3 spatial dimensions.

In the first quantized approach, fermion antisymmetry is encoded through initial
state preparation. Spin labels are also absent and should be accounted for
during state preparation. The cost of initial state preparation is typically
ignored.

In [ ]:
from qualtran import Bloq, CompositeBloq, BloqBuilder, Signature, Register
from qualtran.drawing import show_bloq
from typing import *
import numpy as np
import cirq

## `PrepareFirstQuantizationWithProj`
State preparation for the first quantized chemistry Hamiltonian with a quntum projectile.

#### Parameters
 - `num_bits_p`: The number of bits to represent each dimension of the momentum register.
 - `num_bits_n`: The number of bits to represent each dimension of the momentum register for the projectile. Should be larger than num_bits_p.
 - `eta`: The number of electrons.
 - `num_atoms`: The number of atoms. $L$ in the reference.
 - `lambda_zeta`: sum of nuclear charges.
 - `m_param`: $\mathcal{M}$ in the reference.
 - `num_bits_nuc_pos`: The number of bits of precision for representing the nuclear coordinates.
 - `num_bits_t`: The number of bits of precision for the state preparation over the register selecting between the different components of the Hamiltonian.
 - `num_bits_rot_aa`: The number of bits of precision for the rotation for amplitude amplification.
 - `adjoint`: Whether to dagger the bloq or not. 

#### Registers
 - `tuv`: Flag register for selecting between kinetic and potential terms in the Hamiltonian.
 - `uv`: Flag register for selecting between the different potential components of the Hamiltonian.
 - `i`: A register for selecting electronic registers.
 - `j`: A register for selecting electronic registers.
 - `w`: A register for selecting x, y and z components of the momentum register.
 - `r`: A register for controlling elements of the momentum register. Used for block encodiding kinetic energy operator.
 - `s`: A register for controlling elements of the momentum register. Used for block encodiding kinetic energy operator.
 - `mu`: A register used for implementing nested boxes for the momentum state preparation.
 - `nu_x`: x component of the momentum register for Coulomb potential.
 - `nu_y`: y component of the momentum register for Coulomb potential.
 - `nu_z`: z component of the momentum register for Coulomb potential.
 - `m`: an ancilla register in a uniform superposition.
 - `l`: The register for selecting the nuclei. 

#### References
[Fault-Tolerant Quantum Simulations of Chemistry in First Quantization]( https://arxiv.org/abs/2105.12767)


In [ ]:
from qualtran.bloqs.chemistry.pbc.first_quantization.projectile import PrepareFirstQuantizationWithProj

### Example Instances

In [ ]:
num_bits_p = 6
num_bits_n = 8
eta = 10
num_atoms = 10
lambda_zeta = 10
prep_first_quant = PrepareFirstQuantizationWithProj(
    num_bits_p, num_bits_n, eta, num_atoms, lambda_zeta
)


#### Graphical Signature

In [ ]:
from qualtran.drawing import show_bloqs
show_bloqs([prep_first_quant],
           ['`prep_first_quant`'])

## `SelectFirstQuantizationWithProj`
SELECT operation for the first quantized chemistry Hamiltonian.

#### Parameters
 - `num_bits_p`: The number of bits to represent each dimension of the momentum register.
 - `eta`: The number of electrons.
 - `num_atoms`: The number of atoms. $L$ in the reference.
 - `lambda_zeta`: sum of nuclear charges.
 - `m_param`: $\mathcal{M}$ in the reference.
 - `num_bits_nuc_pos`: The number of bits of precision for representing the nuclear coordinates.
 - `num_bits_t`: The number of bits of precision for the state preparation over the register selecting between the different components of the Hamiltonian.
 - `num_bits_rot_aa`: The number of bits of precision for the rotation for amplitude amplification.
 - `adjoint`: Whether to dagger the bloq or not. 

#### Registers
 - `tuv`: Flag register for selecting between kinetic and potential terms in the Hamiltonian.
 - `uv`: Flag register for selecting between the different potential components of the Hamiltonian.
 - `i_ne_j`: Register flagging $i \ne j$
 - `plus_t`: A register prepared in the $|+\rangle$ state.
 - `i`: A register for selecting electronic registers.
 - `j`: A register for selecting electronic registers.
 - `w`: A register for selecting x, y and z components of the momentum register.
 - `r`: A register for controlling elements of the momentum register. Used for block encodiding kinetic energy operator.
 - `s`: A register for controlling elements of the momentum register. Used for block encodiding kinetic energy operator.
 - `mu`: A register used for implementing nested boxes for the momentum state preparation.
 - `nu_x`: x component of the momentum register for Coulomb potential.
 - `nu_y`: y component of the momentum register for Coulomb potential.
 - `nu_z`: z component of the momentum register for Coulomb potential.
 - `m`: an ancilla register in a uniform superposition.
 - `l`: The register for selecting the nuclei.
 - `sys`: The system register. Will store $\eta$ registers (x, y and z) compents of size num_bits_p. 

#### References
[Fault-Tolerant Quantum Simulations of Chemistry in First Quantization]( https://arxiv.org/abs/2105.12767)


In [ ]:
from qualtran.bloqs.chemistry.pbc.first_quantization.projectile import SelectFirstQuantizationWithProj

### Example Instances

In [ ]:
num_bits_p = 6
num_bits_n = 8
eta = 10
num_atoms = 10
lambda_zeta = 10
sel_first_quant = SelectFirstQuantizationWithProj(
    num_bits_p, num_bits_n, eta, num_atoms, lambda_zeta
)


#### Graphical Signature

In [ ]:
from qualtran.drawing import show_bloqs
show_bloqs([sel_first_quant],
           ['`sel_first_quant`'])